# Bestandsabfrage von Judaica Zeitschriften 
über die DNB SRU Schnittstelle

nutzt MARC21-XML (https://www.loc.gov/marc/bibliographic/)


In [1]:
import pandas as pd
import numpy as np
import os

# 2. Get the metadata

In [2]:
df_metatdata = pd.read_excel('metadata/CM_Seiten_Metadaten.xlsx')
df_metatdata.head()

,VLID_Seite,OT_PATH,VLID_Zs,VLID_Parent,Parent Knotentyp,Parent-Type,Zs_Caption,Volume_Caption,Heft_Caption,Aufsatz_Caption,Datum,Seite (OT_SORT),Seite_Caption,Seitenzahl_Caption
0,52598,|52596||9443938||52598|,52596,9443938,Metadaten,journal volume,... Bericht ueber den Verein für die Provinz W...,8 (1835),NaN,NaN,1835-01-01,1,[Seite],NaN
1,52599,|52596||9443938||52599|,52596,9443938,Metadaten,journal volume,... Bericht ueber den Verein für die Provinz W...,8 (1835),NaN,NaN,1835-01-01,2,[Seite],NaN
2,52600,|52596||9443938||52600|,52596,9443938,Metadaten,journal volume,... Bericht ueber den Verein für die Provinz W...,8 (1835),NaN,NaN,1835-01-01,3,[Seite],NaN
3,52601,|52596||9443938||52601|,52596,9443938,Metadaten,journal volume,... Bericht ueber den Verein für die Provinz W...,8 (1835),NaN,NaN,1835-01-01,4,Seite 4,4
4,52602,|52596||9443938||52602|,52596,9443938,Metadaten,journal volume,... Bericht ueber den Verein für die Provinz W...,8 (1835),NaN,NaN,1835-01-01,5,Seite 5,5


## 

In [3]:
df_metatdata.shape


(918165, 14)

In [4]:
df_metatdata.sample(5)

,VLID_Seite,OT_PATH,VLID_Zs,VLID_Parent,Parent Knotentyp,Parent-Type,Zs_Caption,Volume_Caption,Heft_Caption,Aufsatz_Caption,Datum,Seite (OT_SORT),Seite_Caption,Seitenzahl_Caption
146620,2490775,|2446951||2448608||2448645||2490775|,2446951,2448645,Metadaten,journal issue,Der Israelit,8 (1835),Heft 38-39 (14.5.1896),NaN,1896-05-14,25,NaN,(null)
384554,2890001,|2829132||2829827||2829831||2889910||2890001|,2829132,2889910,Metadaten,article,Monatsschrift für Geschichte und Wissenschaft ...,8 (1835),Heft 3,Beiträge zur Geschichte und Literatur im gaonä...,1910-01-01,14,Seite 318,318
63474,2326179,|2316602||2316663||2316670||2326081||2326179|,2316602,2326081,Strukturknoten,chapter,Im deutschen Reich,8 (1835),Heft 6-7 (Juni 1900),[Ohne Titel] / A. L.,1900-06-01,4,NaN,328
296645,2683242,|2651273||2653452||2653540||2683242|,2651273,2653540,Metadaten,journal issue,Jüdische Rundschau,8 (1835),Heft 95 (27.11.1934),NaN,1934-11-27,2,NaN,(null)
704577,4797693,|4795427||8774843||8800916||4797693|,4795427,8800916,Metadaten,journal issue,Literarishe bleter,8 (1835),28 (15.7.1927),NaN,1927-07-15,6,Seite 534,534


In [5]:
# pandas get all headers
df_metatdata.columns

Index(['VLID_Seite', 'OT_PATH', 'VLID_Zs', 'VLID_Parent', 'Parent Knotentyp',
       'Parent-Type', 'Zs_Caption', 'Volume_Caption', 'Heft_Caption',
       'Aufsatz_Caption', 'Datum', 'Seite (OT_SORT)', 'Seite_Caption',
       'Seitenzahl_Caption'],
      dtype='object')

## Bereinigung der Daten

In [ ]:
# drop columns VLID Seite, OT_PAHT, VLID_Zs
df_metatdata.drop(['VLID_Seite', 'OT_PATH', 'VLID_Zs', 'VLID_Parent', 'Parent Knotentyp', 'Seite (OT_SORT)', 'Seite_Caption', 'Seitenzahl_Caption'], axis=1, inplace=True)

In [ ]:
# list all values in column 'Parent-Type'
df_metatdata['Parent-Type'].unique()


In [ ]:
df_metatdata['Zs_Caption'].unique()

In [ ]:
# remove duplicates in column 'Zs_Caption'
df_metatdata['Zs_Caption'] = df_metatdata['Zs_Caption'].str.replace(' \(.*\)', '')

In [ ]:
df_metatdata['Zs_Caption'].unique()

In [ ]:
df_metatdata.sample(5)

In [ ]:
df_metatdata['Aufsatz_Caption'].unique()

In [ ]:
# load a pickle file
import pickle

# Open the Pickle file
with open('metadata/journal_metadata/journal_list.pickle', 'rb') as file:
    # Load the Pickle data
    loaded_data = pickle.load(file)

# Close the file
file.close()

# Now you can work with the loaded data
print(loaded_data)


In [ ]:
import json

with open('metadata/journal_metadata/journal_metadata_title_lang.json', 'r') as file:
    data = json.load(file)

In [ ]:
import pprint
pprint.pprint(data)

# DNB SRU

In [1]:
import requests
from bs4 import BeautifulSoup as soup
import unicodedata
from lxml import etree
import pandas as pd


In [2]:
def dnb_sru(query, library='dnb'):
    base_url = "https://services.dnb.de/sru/" + library
    params = {'recordSchema' : 'MARC21-xml',
              'operation': 'searchRetrieve',
              'version': '1.1',
              'maximumRecords': '1000',
              'query': query
              }
    r = requests.get(base_url, params=params)
    xml = soup(r.content)
    records = xml.find_all('record', {'type':'Bibliographic'})

    if len(records) < 100:

        return records

    else:

        num_results = 100
        i = 101
        while num_results == 100:

            params.update({'startRecord': i})
            r = requests.get(base_url, params=params)
            xml = soup(r.content)
            new_records = xml.find_all('record', {'type':'Bibliographic'})
            records+=new_records
            i+=100
            num_results = len(new_records)

        return records

In [3]:
records = dnb_sru('tit=Judaica')
print(len(records), 'Ergebnisse')

/Users/benni/.venv/lib/python3.11/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


1393 Ergebnisse


In [4]:
import xml.dom.minidom
temp = soup(open(records), "xml")
new_xml = temp.prettify()
print(new_xml)

TypeError: unhashable type: 'ResultSet'

In [ ]:
def parse_record(record):

    ns = {"marc":"http://www.loc.gov/MARC21/slim"}
    xml = etree.fromstring(unicodedata.normalize("NFC", str(record)))

    #idn
    idn = xml.xpath("marc:controlfield[@tag = '001']", namespaces=ns)
    try:
        idn = idn[0].text
    except:
        idn = 'fail'

    # titel
    title = xml.xpath("marc:datafield[@tag = '245']/marc:subfield[@code = 'a']", namespaces=ns)
    subtitle = xml.xpath("marc:datafield[@tag = '245']/marc:subfield[@code = 'b']", namespaces=ns)

    # category
    category = xml.xpath("marc:datafield[@tag = '084']/marc:subfield[@code = 'a']", namespaces=ns)

    # datum
    date = xml.xpath("marc:datafield[@tag = '264']/marc:subfield[@code = 'c']", namespaces=ns)

    # person 
    person = xml.xpath("marc:datafield[@tag = '100']/marc:subfield[@code = 'a']", namespaces=ns)
    # ddc
    ddc = xml.xpath("marc:datafield[@tag = '082']/marc:subfield[@code= 'a']", namespaces=ns)
    #issn
    issn = xml.xpath("marc:datafield[@tag = '022']/marc:subfield[@code= 'a']", namespaces=ns)

    try:
        title = title[0].text
        subtitle = subtitle[0].text if len(subtitle) > 0 else ""
        category = category[0].text if len(category) > 0 else ""
        date = date[0].text if len(date) > 0 else ""
        person = person[0].text if len(person) > 0 else ""
        ddc = ddc[0].text if len(ddc) > 0 else ""
        issn = issn[0].text if len(issn) > 0 else ""
        #titel = unicodedata.normalize("NFC", titel)
    except:
        titel = "unkown"

    meta_dict = {"idn":idn,
                 'library':library,
                 "title":title,
                 "subtitle":subtitle,
                 "category":category,
                 "date":date,
                 "person":person,
                 'ddc':ddc,
                 'issn':issn}

    return meta_dict

In [ ]:
output = [parse_record(record) for record in records]
df_test = pd.DataFrame(output)
df_test

In [ ]:
# remove the temp_data.pkl file
if os.path.exists('temp_data.pkl'):
    os.remove('temp_data.pkl')

In [ ]:
# print Zs_Caption unique values
titles = df_metatdata['Zs_Caption'].unique()

In [ ]:
print(titles)

In [ ]:
import warnings

warnings.filterwarnings('ignore')
# go through all the titles in the column 'Zs_Caption'
# Load an existing temporary DataFrame if it exists

try:
    temp_df = pd.read_pickle('temp_data.pkl')
except FileNotFoundError:
    temp_df = pd.DataFrame()
    
counter = 1
length = len(titles)

temp_df = pd.DataFrame()
library = 'dnb'
column_headers = ['library', 'idn',  'title', 'subtitle', 'category', 'date', 'person', 'ddc', 'issn']

for title in titles:
    # search for the title in the sru interface
    print('Counter: ' + str(counter) + ' of ' + str(length))
    # calculate the percentage of the progress
    percentage = round(counter / length * 100, 2)
    print('Progress: ' + str(percentage) + '%')
    print('Searching for: ' + title)
    records = dnb_sru('tit=' + title, library=library)
    print(len(records), 'Ergebnisse')
    
    if len(records) > 0:

        output = [parse_record(record) for record in records]
        df_records = pd.DataFrame(output)
        temp_df = pd.concat([temp_df, df_records], ignore_index=True)
    counter = counter + 1
    # Save the temporary DataFrame to a Pickle file
    temp_df.to_pickle('temp_data.pkl')

In [ ]:
import warnings

warnings.filterwarnings('ignore')
# go through all the titles in the column 'Zs_Caption'
# Load an existing temporary DataFrame if it exists

try:
    temp_df = pd.read_pickle('temp_data.pkl')
except FileNotFoundError:
    temp_df = pd.DataFrame()

counter = 1
length = len(titles)

temp_df = pd.DataFrame()
library = 'zdb'

for title in titles:
    # search for the title in the sru interface
    print('Counter: ' + str(counter) + ' of ' + str(length))
    # calculate the percentage of the progress
    percentage = round(counter / length * 100, 2)
    print('Progress: ' + str(percentage) + '%')
    print('Searching for: ' + title)
    records = dnb_sru('tit=' + title, library=library)
    print(len(records), 'Ergebnisse')

    if len(records) > 0:
        output = [parse_record(record) for record in records]
        df_records = pd.DataFrame(output)
        temp_df = pd.concat([temp_df, df_records], ignore_index=True)
    counter = counter + 1
    # Save the temporary DataFrame to a Pickle file
    temp_df.to_pickle('temp_data.pkl')

In [ ]:
large_df = pd.DataFrame()
# Load the temporary DataFrame from the Pickle file
temp_df = pd.read_pickle('temp_data.pkl')

# Append the temporary DataFrame to your large DataFrame
large_df = pd.concat([large_df, temp_df], ignore_index=True)

# remove duplicates
large_df.drop_duplicates(subset=['title', 'date'], keep='first', inplace=True)

# Save the large DataFrame to a CSV file
large_df.to_csv('final_data.csv', index=True, sep=';')

large_df.to_excel('final_data.xlsx')

# Ideen, Fragen

Daten bereinigen
Wie kann ich die Anfrage präzisieren?
1. Kategorie rausfiltern, nur Judaica
2. Exaktes Matching vom Titel - z.B. Jahresbericht.

Herangehensweise 1
Bestehenden Bestand ergänzen.
2. Neue Zeitschriften aus dem Bereich Judacia oder bestimmten 

Gesamten Judaica Bestand abfragen.
1. Zeitschriften
2. Monografien

Über OAI Schnittstelle Sammlungen abfragen


# Ontologie erstellen

Typische Ontoilogie für Zeitschriften, vollständig für unsere Datenbank
Titel
Datum von bis 





# Statistische Auswertungen

Welche Daten konnten vervollständigt werden


# ZDB (Zeitschriftendatenbank)
